<a href="https://colab.research.google.com/github/yoshi2210/SoccerAnalytics/blob/main/Soccer_Tracking_data_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First of all

## Purpose
This code was intended to demo how to use 2D soccer ground positional tracking data, that is homographi'ed' camera movie data , by using Metrica data.

## Note
Metrica data has off-ball positional data, so we can consider it ideal data given camera wouldn't be able to capture all player at once everytime. Skill corner data could be more realitic due to its limitation to which players are on screen at that time. In other words, it's more like on-ball positional data. In this time, I adopt Metrica data because we can find good referencial code and it matches event data too.

In [ ]:
# hogehoge from vscode
import pandas as pd

# GIT

In [ ]:
! git clone https://github.com/metrica-sports/sample-data.git

Cloning into 'sample-data'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 63 (delta 0), reused 0 (delta 0), pack-reused 60
Unpacking objects: 100% (63/63), done.


In [ ]:
!git clone https://github.com/DonsetPG/narya.git

Cloning into 'narya'...
remote: Enumerating objects: 440, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 440 (delta 68), reused 64 (delta 64), pack-reused 360
Receiving objects: 100% (440/440), 24.25 MiB | 29.59 MiB/s, done.
Resolving deltas: 100% (200/200), done.


In [ ]:
! git clone https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking.git

Cloning into 'LaurieOnTracking'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 96 (delta 0), reused 1 (delta 0), pack-reused 92
Unpacking objects: 100% (96/96), done.


# Loading Source data

In [ ]:
# set up initial path to data
#DATADIR = '/PATH/TO/WHERE/YOU/SAVED/THE/SAMPLE/DATA'
#DATADIR = '/Users/laurieshaw/Documents/Football/Data/TrackingData/Metrica/sample-data-master/data'
DATADIR = '/content/sample-data/data'

In [ ]:
# google colabo dir of git clone
from os import chdir as cd
cd('/content/LaurieOnTracking')

In [ ]:
ls

EPV_grid.csv    Metrica_PitchControl.py  Tutorial1_GettingStarted.py
LICENSE         Metrica_Velocities.py    Tutorial2_DelvingDeeper.py
Metrica_EPV.py  Metrica_Viz.py           Tutorial3_PitchControl.py
Metrica_IO.py   README.md                Tutorial4_EPV.py


In [ ]:
import Metrica_IO as mio
import Metrica_Viz as mviz
import Metrica_Velocities as mvel
import Metrica_PitchControl as mpc
import numpy as np
import matplotlib.pyplot as plt

game_id = 1 # let's look at sample match 2

# read in the event data
events = mio.read_event_data(DATADIR,game_id)

# read in tracking data
tracking_home = mio.tracking_data(DATADIR,game_id,'Home')
tracking_away = mio.tracking_data(DATADIR,game_id,'Away')



Reading team: home
Reading team: away


In [ ]:
tracking_home.head(2)

# DataClensing&Enrichment

## Convert positions from metrica units to meters (note change in Metrica's coordinate system since the last lesson)

In [ ]:
# Convert positions from metrica units to meters (note change in Metrica's coordinate system since the last lesson)
tracking_home = mio.to_metric_coordinates(tracking_home)
tracking_away = mio.to_metric_coordinates(tracking_away)
events = mio.to_metric_coordinates(events)


## Flip coordinates in second half so that each team always shoots in the same direction through the match

In [ ]:
#  Flip coordinates in second half so that each team always shoots in the same direction through the match
print('tracking_',tracking_home.shape)
print('event_',events.shape)

for team in [tracking_home,tracking_away,events]:
    print('team_',team.shape)
    second_half_idx = team.Period.idxmax()
    print('second_half',second_half_idx)
    columns = [c for c in team.columns if c[-1].lower() in ['x','y']]
    team.loc[second_half_idx:,columns] *= -1

tracking_ (145006, 32)
event_ (1745, 14)
team_ (145006, 32)
second_half 71269
team_ (145006, 32)
second_half 71269
team_ (1745, 14)
second_half 892


## Add verocities per players

In [ ]:
# calculate speed

import numpy as np
import scipy.signal as signal

def calc_player_velocities(team, smoothing=True, filter_='Savitzky-Golay', window=7, polyorder=1, maxspeed = 12):
    """ calc_player_velocities( tracking_data )
    
    Calculate player velocities in x & y direciton, and total player speed at each timestamp of the tracking data
    
    Parameters
    -----------
        team: the tracking DataFrame for home or away team
        smoothing: boolean variable that determines whether velocity measures are smoothed. Default is True.
        filter: type of filter to use when smoothing the velocities. Default is Savitzky-Golay, which fits a polynomial of order 'polyorder' to the data within each window
        window: smoothing window size in # of frames
        polyorder: order of the polynomial for the Savitzky-Golay filter. Default is 1 - a linear fit to the velcoity, so gradient is the acceleration
        maxspeed: the maximum speed that a player can realisitically achieve (in meters/second). Speed measures that exceed maxspeed are tagged as outliers and set to NaN. 
        
    Returrns
    -----------
       team : the tracking DataFrame with columns for speed in the x & y direction and total speed added

    """
    # remove any velocity data already in the dataframe
    team = remove_player_velocities(team)
    
    # Get the player ids
    player_ids = np.unique( [ c[:-2] for c in team.columns if c[:4] in ['Home','Away'] ] )

    # Calculate the timestep from one frame to the next. Should always be 0.04 within the same half
    dt = team['Time [s]'].diff()
    
    # index of first frame in second half
    second_half_idx = team.Period.idxmax()
    
    # estimate velocities for players in team
    for player in player_ids: # cycle through players individually
        # difference player positions in timestep dt to get unsmoothed estimate of velicity
        vx = team[player+"_x"].diff() / dt
        vy = team[player+"_y"].diff() / dt

        if maxspeed>0:
            # remove unsmoothed data points that exceed the maximum speed (these are most likely position errors)
            raw_speed = np.sqrt( vx**2 + vy**2 )
            vx[ raw_speed>maxspeed ] = np.nan
            vy[ raw_speed>maxspeed ] = np.nan
            
        if smoothing:
            if filter_=='Savitzky-Golay':
                # calculate first half velocity
                vx.loc[:second_half_idx] = signal.savgol_filter(vx.loc[:second_half_idx],window_length=window,polyorder=polyorder)
                vy.loc[:second_half_idx] = signal.savgol_filter(vy.loc[:second_half_idx],window_length=window,polyorder=polyorder)        
                # calculate second half velocity
                vx.loc[second_half_idx:] = signal.savgol_filter(vx.loc[second_half_idx:],window_length=window,polyorder=polyorder)
                vy.loc[second_half_idx:] = signal.savgol_filter(vy.loc[second_half_idx:],window_length=window,polyorder=polyorder)
            elif filter_=='moving average':
                ma_window = np.ones( window ) / window 
                # calculate first half velocity
                vx.loc[:second_half_idx] = np.convolve( vx.loc[:second_half_idx] , ma_window, mode='same' ) 
                vy.loc[:second_half_idx] = np.convolve( vy.loc[:second_half_idx] , ma_window, mode='same' )      
                # calculate second half velocity
                vx.loc[second_half_idx:] = np.convolve( vx.loc[second_half_idx:] , ma_window, mode='same' ) 
                vy.loc[second_half_idx:] = np.convolve( vy.loc[second_half_idx:] , ma_window, mode='same' ) 
                
        
        # put player speed in x,y direction, and total speed back in the data frame
        team[player + "_vx"] = vx
        team[player + "_vy"] = vy
        team[player + "_speed"] = np.sqrt( vx**2 + vy**2 )

    return team

def remove_player_velocities(team):
    # remove player velocoties and acceleeration measures that are already in the 'team' dataframe
    columns = [c for c in team.columns if c.split('_')[-1] in ['vx','vy','ax','ay','speed','acceleration']] # Get the player ids
    team = team.drop(columns=columns)
    return team


# Calculate player velocities
tracking_home = calc_player_velocities(tracking_home,smoothing=True)
tracking_away = calc_player_velocities(tracking_away,smoothing=True)

tracking_home.head(2)

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,...,Home_6_speed,Home_7_vx,Home_7_vy,Home_7_speed,Home_8_vx,Home_8_vy,Home_8_speed,Home_9_vx,Home_9_vy,Home_9_speed
Frame,,,,,,,,,,,,,,,,,,,,,
1,1,0.04,-52.91308,1.19816,-18.39312,-10.41896,-17.27694,0.77316,-20.21738,9.84028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0.08,-52.89824,1.19816,-18.39312,-10.41896,-17.27694,0.77316,-20.21738,9.84028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Self functions

In [ ]:
def find_goalkeeper(team):
    '''
    Find the goalkeeper in team, identifying him/her as the player closest to goal at kick off
    ''' 
    x_columns = [c for c in team.columns if c[-2:].lower()=='_x' and c[:4] in ['Home','Away']]
    # GK_col = team.iloc[0][x_columns].abs().idxmax(axis=1)
    GK_col = team.iloc[0][x_columns].abs().idxmax()
    return GK_col.split('_')[1]

def calculate_pitch_control_at_target(target_position, attacking_players, defending_players, ball_start_pos, params):
    """ calculate_pitch_control_at_target
    
    Calculates the pitch control probability for the attacking and defending teams at a specified target position on the ball.
    
    Parameters
    -----------
        target_position: size 2 numpy array containing the (x,y) position of the position on the field to evaluate pitch control
        attacking_players: list of 'player' objects (see player class above) for the players on the attacking team (team in possession)
        defending_players: list of 'player' objects (see player class above) for the players on the defending team
        ball_start_pos: Current position of the ball (start position for a pass). If set to NaN, function will assume that the ball is already at the target position.
        params: Dictionary of model parameters (default model parameters can be generated using default_model_params() )
        
    Returrns
    -----------
        PPCFatt: Pitch control probability for the attacking team
        PPCFdef: Pitch control probability for the defending team ( 1-PPCFatt-PPCFdef <  params['model_converge_tol'] )

    """
    # calculate ball travel time from start position to end position.
    if ball_start_pos is None or any(np.isnan(ball_start_pos)): # assume that ball is already at location
        ball_travel_time = 0.0 
    else:
        # ball travel time is distance to target position from current ball position divided assumed average ball speed
        ball_travel_time = np.linalg.norm( target_position - ball_start_pos )/params['average_ball_speed']
    
    # first get arrival time of 'nearest' attacking player (nearest also dependent on current velocity)
    tau_min_att = np.nanmin( [p.simple_time_to_intercept(target_position) for p in attacking_players] )
    tau_min_def = np.nanmin( [p.simple_time_to_intercept(target_position ) for p in defending_players] )
    
    # check whether we actually need to solve equation 3
    if tau_min_att-max(ball_travel_time,tau_min_def) >= params['time_to_control_def']:
        # if defending team can arrive significantly before attacking team, no need to solve pitch control model
        return 0., 1.
    elif tau_min_def-max(ball_travel_time,tau_min_att) >= params['time_to_control_att']:
        # if attacking team can arrive significantly before defending team, no need to solve pitch control model
        return 1., 0.
    else: 
        # solve pitch control model by integrating equation 3 in Spearman et al.
        # first remove any player that is far (in time) from the target location
        attacking_players = [p for p in attacking_players if p.time_to_intercept-tau_min_att < params['time_to_control_att'] ]
        defending_players = [p for p in defending_players if p.time_to_intercept-tau_min_def < params['time_to_control_def'] ]
        # set up integration arrays
        dT_array = np.arange(ball_travel_time-params['int_dt'],ball_travel_time+params['max_int_time'],params['int_dt']) 
        PPCFatt = np.zeros_like( dT_array )
        PPCFdef = np.zeros_like( dT_array )
        # integration equation 3 of Spearman 2018 until convergence or tolerance limit hit (see 'params')
        ptot = 0.0
        i = 1
        while 1-ptot>params['model_converge_tol'] and i<dT_array.size: 
            T = dT_array[i]
            for player in attacking_players:
                # calculate ball control probablity for 'player' in time interval T+dt
                dPPCFdT = (1-PPCFatt[i-1]-PPCFdef[i-1])*player.probability_intercept_ball( T ) * player.lambda_att
                # make sure it's greater than zero
                assert dPPCFdT>=0, 'Invalid attacking player probability (calculate_pitch_control_at_target)'
                player.PPCF += dPPCFdT*params['int_dt'] # total contribution from individual player
                PPCFatt[i] += player.PPCF # add to sum over players in the attacking team (remembering array element is zero at the start of each integration iteration)
            for player in defending_players:
                # calculate ball control probablity for 'player' in time interval T+dt
                dPPCFdT = (1-PPCFatt[i-1]-PPCFdef[i-1])*player.probability_intercept_ball( T ) * player.lambda_def
                # make sure it's greater than zero
                assert dPPCFdT>=0, 'Invalid defending player probability (calculate_pitch_control_at_target)'
                player.PPCF += dPPCFdT*params['int_dt'] # total contribution from individual player
                PPCFdef[i] += player.PPCF # add to sum over players in the defending team
            ptot = PPCFdef[i]+PPCFatt[i] # total pitch control probability 
            i += 1
        if i>=dT_array.size:
            print("Integration failed to converge: %1.3f" % (ptot) )
        return PPCFatt[i-1], PPCFdef[i-1]



class player(object):
    """
    player() class
    
    Class defining a player object that stores position, velocity, time-to-intercept and pitch control contributions for a player
    
    __init__ Parameters
    -----------
    pid: id (jersey number) of player
    team: row of tracking data for team
    teamname: team name "Home" or "Away"
    params: Dictionary of model parameters (default model parameters can be generated using default_model_params() )
    

    methods include:
    -----------
    simple_time_to_intercept(r_final): time take for player to get to target position (r_final) given current position
    probability_intercept_ball(T): probability player will have controlled ball at time T given their expected time_to_intercept
    
    """
    # player object holds position, velocity, time-to-intercept and pitch control contributions for each player
    def __init__(self,pid,team,teamname,params,GKid):
        self.id = pid
        self.is_gk = self.id == GKid
        self.teamname = teamname
        self.playername = "%s_%s_" % (teamname,pid)
        self.vmax = params['max_player_speed'] # player max speed in m/s. Could be individualised
        self.reaction_time = params['reaction_time'] # player reaction time in 's'. Could be individualised
        self.tti_sigma = params['tti_sigma'] # standard deviation of sigmoid function (see Eq 4 in Spearman, 2018)
        self.lambda_att = params['lambda_att'] # standard deviation of sigmoid function (see Eq 4 in Spearman, 2018)
        self.lambda_def = params['lambda_gk'] if self.is_gk else params['lambda_def'] # factor of 3 ensures that anything near the GK is likely to be claimed by the GK
        self.get_position(team)
        self.get_velocity(team)
        self.PPCF = 0. # initialise this for later
        
    def get_position(self,team):
        self.position = np.array( [ team[self.playername+'x'], team[self.playername+'y'] ] )
        self.inframe = not np.any( np.isnan(self.position) )
        
    def get_velocity(self,team):
        self.velocity = np.array( [ team[self.playername+'vx'], team[self.playername+'vy'] ] )
        if np.any( np.isnan(self.velocity) ):
            self.velocity = np.array([0.,0.])
    
    def simple_time_to_intercept(self, r_final):
        self.PPCF = 0. # initialise this for later
        # Time to intercept assumes that the player continues moving at current velocity for 'reaction_time' seconds
        # and then runs at full speed to the target position.
        r_reaction = self.position + self.velocity*self.reaction_time
        self.time_to_intercept = self.reaction_time + np.linalg.norm(r_final-r_reaction)/self.vmax
        return self.time_to_intercept

    def probability_intercept_ball(self,T):
        # probability of a player arriving at target location at time 'T' given their expected time_to_intercept (time of arrival), as described in Spearman 2018
        f = 1/(1. + np.exp( -np.pi/np.sqrt(3.0)/self.tti_sigma * (T-self.time_to_intercept) ) )
        return f


def initialise_players(team,teamname,params,GKid):
    """
    initialise_players(team,teamname,params)
    
    create a list of player objects that holds their positions and velocities from the tracking data dataframe 
    
    Parameters
    -----------
    
    team: row (i.e. instant) of either the home or away team tracking Dataframe
    teamname: team name "Home" or "Away"
    params: Dictionary of model parameters (default model parameters can be generated using default_model_params() )
        
    Returns
    -----------
    
    team_players: list of player objects for the team at at given instant
    
    """    
    # get player  ids
    player_ids = np.unique( [ c.split('_')[1] for c in team.keys() if c[:4] == teamname ] )
    # create list
    team_players = []
    for p in player_ids:
        # create a player object for player_id 'p'
        team_player = player(p,team,teamname,params,GKid)
        if team_player.inframe:
            team_players.append(team_player)
    return team_players

def generate_pitch_control_for_track(certain_frame, tracking_home, tracking_away, params, GK_numbers, field_dimen = (106.,68.,), n_grid_cells_x = 50, offsides=False):
    """ generate_pitch_control_for_event
    
    Evaluates pitch control surface over the entire field at the moment of the given event (determined by the index of the event passed as an input)
    
    Parameters
    -----------
        event_id: Index (not row) of the event that describes the instant at which the pitch control surface should be calculated
        events: Dataframe containing the event data
        tracking_home: tracking DataFrame for the Home team
        tracking_away: tracking DataFrame for the Away team
        params: Dictionary of model parameters (default model parameters can be generated using default_model_params() )
        GK_numbers: tuple containing the player id of the goalkeepers for the (home team, away team)
        field_dimen: tuple containing the length and width of the pitch in meters. Default is (106,68)
        n_grid_cells_x: Number of pixels in the grid (in the x-direction) that covers the surface. Default is 50.
                        n_grid_cells_y will be calculated based on n_grid_cells_x and the field dimensions
        offsides: If True, find and remove offside atacking players from the calculation. Default is True.
        
    UPDATE (tutorial 4): Note new input arguments ('GK_numbers' and 'offsides')
        
    Returrns
    -----------
        PPCFa: Pitch control surface (dimen (n_grid_cells_x,n_grid_cells_y) ) containing pitch control probability for the attcking team.
               Surface for the defending team is just 1-PPCFa.
        xgrid: Positions of the pixels in the x-direction (field length)
        ygrid: Positions of the pixels in the y-direction (field width)

    """
    # get the details of the event (frame, team in possession, ball_start_position)
    # pass_frame = events.loc[event_id]['Start Frame']
    pass_frame = certain_frame
    
    #pass_team = events.loc[event_id].Team
    #ball_start_pos = np.array([events.loc[event_id]['Start X'],events.loc[event_id]['Start Y']])
    
    ball_start_pos = np.array([tracking_home.ball_x.loc[certain_frame],tracking_home.ball_y.loc[certain_frame]])

    # break the pitch down into a grid
    n_grid_cells_y = int(n_grid_cells_x*field_dimen[1]/field_dimen[0])
    dx = field_dimen[0]/n_grid_cells_x
    dy = field_dimen[1]/n_grid_cells_y
    xgrid = np.arange(n_grid_cells_x)*dx - field_dimen[0]/2. + dx/2.
    ygrid = np.arange(n_grid_cells_y)*dy - field_dimen[1]/2. + dy/2.
    # initialise pitch control grids for attacking and defending teams 
    PPCFa = np.zeros( shape = (len(ygrid), len(xgrid)) )
    PPCFd = np.zeros( shape = (len(ygrid), len(xgrid)) )
    # initialise player positions and velocities for pitch control calc (so that we're not repeating this at each grid cell position)
    # if pass_team=='Home':
    #     attacking_players = initialise_players(tracking_home.loc[pass_frame],'Home',params,GK_numbers[0])
    #     defending_players = initialise_players(tracking_away.loc[pass_frame],'Away',params,GK_numbers[1])
    # elif pass_team=='Away':
    #     defending_players = initialise_players(tracking_home.loc[pass_frame],'Home',params,GK_numbers[0])
    #     attacking_players = initialise_players(tracking_away.loc[pass_frame],'Away',params,GK_numbers[1])
    # else:
    #     assert False, "Team in possession must be either home or away"
        
    # Assumption, either is ok
    attacking_players = initialise_players(tracking_home.loc[pass_frame],'Home',params,GK_numbers[0])
    defending_players = initialise_players(tracking_away.loc[pass_frame],'Away',params,GK_numbers[1])


    # find any attacking players that are offside and remove them from the pitch control calculation
    # if offsides:
    #     attacking_players = check_offsides( attacking_players, defending_players, ball_start_pos, GK_numbers)

    # calculate pitch pitch control model at each location on the pitch
    for i in range( len(ygrid) ):
        for j in range( len(xgrid) ):
            target_position = np.array( [xgrid[j], ygrid[i]] )
            PPCFa[i,j],PPCFd[i,j] = calculate_pitch_control_at_target(target_position, attacking_players, defending_players, ball_start_pos, params)
    # check probabilitiy sums within convergence
    checksum = np.sum( PPCFa + PPCFd ) / float(n_grid_cells_y*n_grid_cells_x ) 
    assert 1-checksum < params['model_converge_tol'], "Checksum failed: %1.3f" % (1-checksum)
    return PPCFa,xgrid,ygrid


def plot_pitchcontrol_for_track( certain_frame, tracking_home, tracking_away, PPCF, alpha = 0.7, include_player_velocities=True, annotate=False, field_dimen = (106.0,68)):
    """ plot_pitchcontrol_for_event( event_id, events,  tracking_home, tracking_away, PPCF )
    
    Plots the pitch control surface at the instant of the event given by the event_id. Player and ball positions are overlaid.
    
    Parameters
    -----------
        event_id: Index (not row) of the event that describes the instant at which the pitch control surface should be calculated
        events: Dataframe containing the event data
        tracking_home: (entire) tracking DataFrame for the Home team
        tracking_away: (entire) tracking DataFrame for the Away team
        PPCF: Pitch control surface (dimen (n_grid_cells_x,n_grid_cells_y) ) containing pitch control probability for the attcking team (as returned by the generate_pitch_control_for_event in Metrica_PitchControl)
        alpha: alpha (transparency) of player markers. Default is 0.7
        include_player_velocities: Boolean variable that determines whether player velocities are also plotted (as quivers). Default is False
        annotate: Boolean variable that determines with player jersey numbers are added to the plot (default is False)
        field_dimen: tuple containing the length and width of the pitch in meters. Default is (106,68)
        
    NB: this function no longer requires xgrid and ygrid as an input
        
    Returrns
    -----------
       fig,ax : figure and aixs objects (so that other data can be plotted onto the pitch)

    """    

    # pick a pass at which to generate the pitch control surface
    # pass_frame = events.loc[event_id]['Start Frame']
    pass_frame = certain_frame
    #pass_team = events.loc[event_id].Team
    # either is ok
    pass_team = 'Home'

    # plot frame and event
    fig,ax = plot_pitch(field_color='white', field_dimen = field_dimen)
    plot_frame( tracking_home.loc[pass_frame], tracking_away.loc[pass_frame], figax=(fig,ax), PlayerAlpha=alpha, include_player_velocities=include_player_velocities, annotate=annotate )
    #plot_events( events.loc[event_id:event_id], figax = (fig,ax), indicators = ['Marker','Arrow'], annotate=False, color= 'k', alpha=1 )
    
    # plot pitch control surface
    if pass_team=='Home':
        cmap = 'bwr'
    else:
        cmap = 'bwr_r'
    ax.imshow(np.flipud(PPCF), extent=(-field_dimen[0]/2., field_dimen[0]/2., -field_dimen[1]/2., field_dimen[1]/2.),interpolation='spline36',vmin=0.0,vmax=1.0,cmap=cmap,alpha=0.5)

    save_filename = '/content/PPCF_'+str(certain_frame)+'.png'
    fig.savefig(save_filename)

    return fig,ax


def plot_pitch( field_dimen = (106.0,68.0), field_color ='green', linewidth=2, markersize=20):
    """ plot_pitch
    
    Plots a soccer pitch. All distance units converted to meters.
    
    Parameters
    -----------
        field_dimen: (length, width) of field in meters. Default is (106,68)
        field_color: color of field. options are {'green','white'}
        linewidth  : width of lines. default = 2
        markersize : size of markers (e.g. penalty spot, centre spot, posts). default = 20
        
    Returrns
    -----------
       fig,ax : figure and aixs objects (so that other data can be plotted onto the pitch)

    """
    fig,ax = plt.subplots(figsize=(12,8)) # create a figure 
    # decide what color we want the field to be. Default is green, but can also choose white
    if field_color=='green':
        ax.set_facecolor('mediumseagreen')
        lc = 'whitesmoke' # line color
        pc = 'w' # 'spot' colors
    elif field_color=='white':
        lc = 'k'
        pc = 'k'
    # ALL DIMENSIONS IN m
    border_dimen = (3,3) # include a border arround of the field of width 3m
    meters_per_yard = 0.9144 # unit conversion from yards to meters
    half_pitch_length = field_dimen[0]/2. # length of half pitch
    half_pitch_width = field_dimen[1]/2. # width of half pitch
    signs = [-1,1] 
    # Soccer field dimensions typically defined in yards, so we need to convert to meters
    goal_line_width = 8*meters_per_yard
    box_width = 20*meters_per_yard
    box_length = 6*meters_per_yard
    area_width = 44*meters_per_yard
    area_length = 18*meters_per_yard
    penalty_spot = 12*meters_per_yard
    corner_radius = 1*meters_per_yard
    D_length = 8*meters_per_yard
    D_radius = 10*meters_per_yard
    D_pos = 12*meters_per_yard
    centre_circle_radius = 10*meters_per_yard
    # plot half way line # center circle
    ax.plot([0,0],[-half_pitch_width,half_pitch_width],lc,linewidth=linewidth)
    ax.scatter(0.0,0.0,marker='o',facecolor=lc,linewidth=0,s=markersize)
    y = np.linspace(-1,1,50)*centre_circle_radius
    x = np.sqrt(centre_circle_radius**2-y**2)
    ax.plot(x,y,lc,linewidth=linewidth)
    ax.plot(-x,y,lc,linewidth=linewidth)
    for s in signs: # plots each line seperately
        # plot pitch boundary
        ax.plot([-half_pitch_length,half_pitch_length],[s*half_pitch_width,s*half_pitch_width],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length,s*half_pitch_length],[-half_pitch_width,half_pitch_width],lc,linewidth=linewidth)
        # goal posts & line
        ax.plot( [s*half_pitch_length,s*half_pitch_length],[-goal_line_width/2.,goal_line_width/2.],pc+'s',markersize=6*markersize/20.,linewidth=linewidth)
        # 6 yard box
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*box_length],[box_width/2.,box_width/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*box_length],[-box_width/2.,-box_width/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length-s*box_length,s*half_pitch_length-s*box_length],[-box_width/2.,box_width/2.],lc,linewidth=linewidth)
        # penalty area
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*area_length],[area_width/2.,area_width/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*area_length],[-area_width/2.,-area_width/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length-s*area_length,s*half_pitch_length-s*area_length],[-area_width/2.,area_width/2.],lc,linewidth=linewidth)
        # penalty spot
        ax.scatter(s*half_pitch_length-s*penalty_spot,0.0,marker='o',facecolor=lc,linewidth=0,s=markersize)
        # corner flags
        y = np.linspace(0,1,50)*corner_radius
        x = np.sqrt(corner_radius**2-y**2)
        ax.plot(s*half_pitch_length-s*x,-half_pitch_width+y,lc,linewidth=linewidth)
        ax.plot(s*half_pitch_length-s*x,half_pitch_width-y,lc,linewidth=linewidth)
        # draw the D
        y = np.linspace(-1,1,50)*D_length # D_length is the chord of the circle that defines the D
        x = np.sqrt(D_radius**2-y**2)+D_pos
        ax.plot(s*half_pitch_length-s*x,y,lc,linewidth=linewidth)
        
    # remove axis labels and ticks
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    # set axis limits
    xmax = field_dimen[0]/2. + border_dimen[0]
    ymax = field_dimen[1]/2. + border_dimen[1]
    ax.set_xlim([-xmax,xmax])
    ax.set_ylim([-ymax,ymax])
    ax.set_axisbelow(True)
    return fig,ax


def plot_frame( hometeam, awayteam, figax=None, team_colors=('r','b'), field_dimen = (106.0,68.0), include_player_velocities=False, PlayerMarkerSize=10, PlayerAlpha=0.7, annotate=False ):
    """ plot_frame( hometeam, awayteam )
    
    Plots a frame of Metrica tracking data (player positions and the ball) on a football pitch. All distances should be in meters.
    
    Parameters
    -----------
        hometeam: row (i.e. instant) of the home team tracking data frame
        awayteam: row of the away team tracking data frame
        fig,ax: Can be used to pass in the (fig,ax) objects of a previously generated pitch. Set to (fig,ax) to use an existing figure, or None (the default) to generate a new pitch plot, 
        team_colors: Tuple containing the team colors of the home & away team. Default is 'r' (red, home team) and 'b' (blue away team)
        field_dimen: tuple containing the length and width of the pitch in meters. Default is (106,68)
        include_player_velocities: Boolean variable that determines whether player velocities are also plotted (as quivers). Default is False
        PlayerMarkerSize: size of the individual player marlers. Default is 10
        PlayerAlpha: alpha (transparency) of player markers. Defaault is 0.7
        annotate: Boolean variable that determines with player jersey numbers are added to the plot (default is False)
        
    Returrns
    -----------
       fig,ax : figure and aixs objects (so that other data can be plotted onto the pitch)

    """
    if figax is None: # create new pitch 
        fig,ax = plot_pitch( field_dimen = field_dimen )
    else: # overlay on a previously generated pitch
        fig,ax = figax # unpack tuple
    # plot home & away teams in order
    for team,color in zip( [hometeam,awayteam], team_colors) :
        x_columns = [c for c in team.keys() if c[-2:].lower()=='_x' and c!='ball_x'] # column header for player x positions
        y_columns = [c for c in team.keys() if c[-2:].lower()=='_y' and c!='ball_y'] # column header for player y positions
        ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
        if include_player_velocities:
            vx_columns = ['{}_vx'.format(c[:-2]) for c in x_columns] # column header for player x positions
            vy_columns = ['{}_vy'.format(c[:-2]) for c in y_columns] # column header for player y positions
            ax.quiver( team[x_columns], team[y_columns], team[vx_columns], team[vy_columns], color=color, scale_units='inches', scale=10.,width=0.0015,headlength=5,headwidth=3,alpha=PlayerAlpha)
        if annotate:
            [ ax.text( team[x]+0.5, team[y]+0.5, x.split('_')[1], fontsize=10, color=color  ) for x,y in zip(x_columns,y_columns) if not ( np.isnan(team[x]) or np.isnan(team[y]) ) ] 
    # plot ball
    ax.plot( hometeam['ball_x'], hometeam['ball_y'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
    return fig,ax

# Visualize Pitch Controll



In [ ]:
# first get pitch control model parameters
params = mpc.default_model_params()

# find goalkeepers for offside calculation
GK_numbers = [find_goalkeeper(tracking_home),find_goalkeeper(tracking_away)]

hilighted_event_id=820

# evaluated pitch control surface for first pass
PPCF,xgrid,ygrid = mpc.generate_pitch_control_for_event(hilighted_event_id, events, tracking_home, tracking_away, params, GK_numbers, field_dimen = (106.,68.,), n_grid_cells_x = 50)
mviz.plot_pitchcontrol_for_event( hilighted_event_id, events,  tracking_home, tracking_away, PPCF, annotate=False )

# evaluated pitch control per frame
PPCF,xgrid,ygrid = generate_pitch_control_for_track(events.iloc[hilighted_event_id]['Start Frame'], tracking_home, tracking_away, params, GK_numbers, field_dimen = (106.,68.,), n_grid_cells_x = 50)
plot_pitchcontrol_for_track( events.iloc[hilighted_event_id]['Start Frame'], tracking_home, tracking_away, PPCF, annotate=False )

# Scene extract

## data run through 

In [ ]:
tracking_home.columns

Index(['Period', 'Time [s]', 'Home_11_x', 'Home_11_y', 'Home_1_x', 'Home_1_y',
       'Home_2_x', 'Home_2_y', 'Home_3_x', 'Home_3_y', 'Home_4_x', 'Home_4_y',
       'Home_5_x', 'Home_5_y', 'Home_6_x', 'Home_6_y', 'Home_7_x', 'Home_7_y',
       'Home_8_x', 'Home_8_y', 'Home_9_x', 'Home_9_y', 'Home_10_x',
       'Home_10_y', 'Home_12_x', 'Home_12_y', 'Home_13_x', 'Home_13_y',
       'Home_14_x', 'Home_14_y', 'ball_x', 'ball_y', 'Home_1_vx', 'Home_1_vy',
       'Home_1_speed', 'Home_10_vx', 'Home_10_vy', 'Home_10_speed',
       'Home_11_vx', 'Home_11_vy', 'Home_11_speed', 'Home_12_vx', 'Home_12_vy',
       'Home_12_speed', 'Home_13_vx', 'Home_13_vy', 'Home_13_speed',
       'Home_14_vx', 'Home_14_vy', 'Home_14_speed', 'Home_2_vx', 'Home_2_vy',
       'Home_2_speed', 'Home_3_vx', 'Home_3_vy', 'Home_3_speed', 'Home_4_vx',
       'Home_4_vy', 'Home_4_speed', 'Home_5_vx', 'Home_5_vy', 'Home_5_speed',
       'Home_6_vx', 'Home_6_vy', 'Home_6_speed', 'Home_7_vx', 'Home_7_vy',
       'Home_7_spe

In [ ]:
tracking_home.head(2)

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,...,Home_6_speed,Home_7_vx,Home_7_vy,Home_7_speed,Home_8_vx,Home_8_vy,Home_8_speed,Home_9_vx,Home_9_vy,Home_9_speed
Frame,,,,,,,,,,,,,,,,,,,,,
1,1,0.04,-52.91308,1.19816,-18.39312,-10.41896,-17.27694,0.77316,-20.21738,9.84028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0.08,-52.89824,1.19816,-18.39312,-10.41896,-17.27694,0.77316,-20.21738,9.84028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## Tracking data plot
pd.set_option('display.max_rows', None)
# print(tracking_home.isnull().sum())

fig,ax = plt.subplots(figsize=(15,8))
for i in range(14):
  y_str = 'Home_'+str(i+1)+'_x'
  plt.plot(tracking_home['Time [s]'], tracking_home[y_str]+np.ones(len(tracking_home[y_str]))*100*i)

plt.plot(tracking_home['Time [s]'], tracking_home['ball_x']+np.ones(len(tracking_home[y_str]))*100*15)

In [ ]:
events.head(2)

,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y
0,Away,SET PIECE,KICK OFF,1,1,0.04,0,0.00,Player19,NaN,NaN,NaN,NaN,NaN
1,Away,PASS,NaN,1,1,0.04,3,0.12,Player19,Player21,-5.3,7.48,5.3,4.76


In [ ]:
Event_type_list = events.Type.unique().tolist()
print(Event_type_list)

fig,ax = plt.subplots(figsize=(15,8))

for i,j in enumerate(Event_type_list):

  plt.plot(events[events['Type']==j]['Start Time [s]'],events[events['Type']==j]['Start X']+np.ones(len(events[events['Type']==j]['Start X']))*100*i,'*',label=j)  

plt.legend(loc="upper left", bbox_to_anchor=(1.02, 1.0,))

In [ ]:
plt.plot(tracking_home['Time [s]'], tracking_home['Home_11_x'],label='x')
plt.plot(tracking_home['Time [s]'], tracking_home['Home_11_y'],label='y')
plt.legend()

In [ ]:
plt.plot(tracking_away['Time [s]'], tracking_away['Away_25_x'],label='x')
plt.plot(tracking_away['Time [s]'], tracking_away['Away_25_y'],label='y')
plt.legend()

In [ ]:
tracking_away.columns

In [ ]:
## Tracking data plot
pd.set_option('display.max_rows', None)
# print(tracking_home.isnull().sum())

fig,ax = plt.subplots(figsize=(15,8))
for i in range(14,28):
  y_str = 'Away_'+str(i+1)+'_x'
  plt.plot(tracking_away['Time [s]'], tracking_away[y_str]+np.ones(len(tracking_away[y_str]))*100*i,label=y_str)

plt.plot(tracking_away['Time [s]'], tracking_away['ball_x']+np.ones(len(tracking_away[y_str]))*100*(29),label='ball')

plt.legend(loc="upper left", bbox_to_anchor=(1.02, 1.0,))

## Long counter 

## Rule consideration

In [ ]:
# How to extract long counter?

# Define a timewindow
# fps_demo = 25
# search_window_time = 10 # [s] 
# search_window_frame = search_window_time*fps_demo
search_window_frame = 15*25 # 15[s]*25 [fps]
windows_start = 6700
windows_end = windows_start+search_window_frame

# Visualize ball track
fig,ax = plt.subplots(figsize=(20,4))
ax.plot(tracking_home[windows_start:windows_end].index,tracking_home[windows_start:windows_end]['ball_x'],'-*')
ax.axvspan(windows_start, windows_end, color='red', alpha=0.5)

fig,ax = plt.subplots(figsize=(20,4))
ax.plot(tracking_home[:15000].index,tracking_home[:15000]['ball_x'])
ax.axvspan(windows_start, windows_end, color='red', alpha=0.5)

# Set a rule -30 to 30 , maxidx-minidx<0
print(tracking_home[windows_start:windows_end]['ball_x'].min())
print(tracking_home[windows_start:windows_end]['ball_x'].max())
print(tracking_home[windows_start:windows_end]['ball_x'].idxmin())
print(tracking_home[windows_start:windows_end]['ball_x'].idxmax())
print('Positive = defence: ',tracking_home[windows_start:windows_end]['ball_x'].idxmax()-tracking_home[windows_start:windows_end]['ball_x'].idxmin())


In [ ]:
time_list = tracking_home['Time [s]'].tolist()
frame_list = tracking_home.index.tolist()
counter_flag_list=[]
window_min_list=[]
window_max_list=[]

for i,frame_value in enumerate(frame_list):
  windows_start = frame_value
  windows_end = windows_start+search_window_frame

  min_threshold = -40
  max_threshold = 40
  threshold = 60

  if(tracking_home[windows_start:windows_end]['ball_x'].min()<min_threshold) and (tracking_home[windows_start:windows_end]['ball_x'].max()>max_threshold) and (tracking_home[windows_start:windows_end]['ball_x'].idxmax()-tracking_home[windows_start:windows_end]['ball_x'].idxmin())<0:
  # if(tracking_home[windows_start:windows_end]['ball_x'].max()-tracking_home[windows_start:windows_end]['ball_x'].min()>threshold) and (tracking_home[windows_start:windows_end]['ball_x'].idxmax()-tracking_home[windows_start:windows_end]['ball_x'].idxmin())>0:
    counter_flag_list.append(1)
    window_min_list.append(tracking_home[windows_start:windows_end]['ball_x'].min())
    window_max_list.append(tracking_home[windows_start:windows_end]['ball_x'].max())
    
  else:
    counter_flag_list.append(0)
    window_min_list.append(tracking_home[windows_start:windows_end]['ball_x'].min())
    window_max_list.append(tracking_home[windows_start:windows_end]['ball_x'].max())



## Visualize methods and results

In [ ]:
from scipy.signal import argrelextrema
from scipy.signal import find_peaks
df_pattern['counter_flag'].unique()

array([0, 1])

In [ ]:
tracking_home.shape[0]

145006

In [ ]:
df_pattern = pd.DataFrame(list(zip(frame_list,time_list,counter_flag_list,window_min_list,window_max_list)),columns=['frame','time','counter_flag','window_min','window_max'])
df_pattern['ball_dif'] =df_pattern['window_max'] -df_pattern['window_min']  
df_pattern['counte_flag_rolling_10seconds'] = df_pattern['counter_flag'].rolling(25*10).mean()
peaks, _ = find_peaks(df_pattern['counte_flag_rolling_10seconds'], height=0)
print(peaks)

# Event detection
fig,ax = plt.subplots(figsize=(20,4))
df_pattern.plot(x='frame',y='counter_flag',ax=ax,style='-',color='red')
df_pattern.plot(x='frame',y='counte_flag_rolling_10seconds',ax=ax,style='-',color='green')
ax.scatter(peaks,df_pattern['counte_flag_rolling_10seconds'][peaks],linewidth=0.3, s=100, c='b')
ax.set_xlim(1,tracking_home.shape[0])

fig,ax = plt.subplots(figsize=(20,4))
ax.plot(tracking_home.index,tracking_home['ball_x'])
for peaks_idx in peaks:
  ax.axvspan(peaks_idx-round(search_window_frame/2), peaks_idx+round(search_window_frame/2), color='red', alpha=0.5)
ax.set_xlim(1,tracking_home.shape[0])

## Picking up movies automatically

In [ ]:
# 
# 9228, 13507, 102571

for slice_frame in peaks:
  movie_length = 20
  output_file_name = 'Slice_'+str(slice_frame)+'th_frame'
  mviz.save_match_clip(tracking_home[slice_frame-25*5:slice_frame+25*movie_length],tracking_away[slice_frame-25*5:slice_frame+25*movie_length], '/content', fname=output_file_name, figax=None, frames_per_second=25, team_colors=('r','b'), field_dimen = (106.0,68.0), include_player_velocities=True, PlayerMarkerSize=10, PlayerAlpha=0.7)

Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done
Generating movie...done


In [ ]:
# Pick up one for creating png figures per slice
nth=0

for i in range(peaks[nth]-25*5,peaks[nth]+25*movie_length):
  # evaluated pitch control per frame
  print(i+1)
  PPCF,xgrid,ygrid = generate_pitch_control_for_track(i+1, tracking_home, tracking_away, params, GK_numbers, field_dimen = (106.,68.,), n_grid_cells_x = 50)
  plot_pitchcontrol_for_track( i+1, tracking_home, tracking_away, PPCF, annotate=False )

In [ ]:
# Merging all png figures to a movie
import cv2
import numpy as np

width=864
height=576
fps=25

# choose codec according to format needed
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('/content/PPCF_Counter.avi', fourcc, fps, (width, height))


# for i in range(start_frame-1,end_frame):
for i in range(peaks[nth]-25*5,peaks[nth]+25*movie_length):
    img = cv2.imread('/content/PPCF_'+str(i+1)+'.png')
    video.write(img)

cv2.destroyAllWindows()
video.release()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# # importing movie py libraries
# from moviepy.editor import VideoClip
# from moviepy.video.io.bindings import mplfig_to_npimage

# #coding:utf-8 importing for pasting an pitch image
# from PIL import Image
# import matplotlib.pyplot as plt
# import numpy as np

# # duration of the video
# duration = 60

# fig,ax = plt.subplots(figsize=(20,4))

# # method to get frames
# def make_frame(t):
#   # clear
#   ax.clear()

#   windows_start_time = t*10
#   windows_end_time = windows_start_time+search_window_time
#   ax.plot(tracking_home[:10*60*25]['Time [s]'],tracking_home[:10*60*25]['ball_x'])
#   ax.axvspan(windows_start_time, windows_end_time, color='red', alpha=0.5)

#   # returning numpy image
#   return mplfig_to_npimage(fig)

# # creating animation
# animation = VideoClip(make_frame, duration = duration)

# # displaying animation with auto play and looping
# animation.ipython_display(fps = 5, loop = True, autoplay = True)

# Create&Save Movies

## Define where to cut and how freq.

In [ ]:
# frame to cut
L = list(range(tracking_home.index.max()))
sparse_num = 1 # int(0.5/0.04)
sparse_L=L[::sparse_num]

# creating a movie 
# start_frame = sparse_L[:20][0]+ 1
# end_frame = sparse_L[:20][len(sparse_L[:20])-1] + 1

start_frame = 25*5
end_frame = 25*5+25*30

## Normal 2D movies

In [ ]:
mviz.save_match_clip(tracking_home[start_frame:end_frame],tracking_away[start_frame:end_frame], '/content', fname='clip_test_v1', figax=None, frames_per_second=25, team_colors=('r','b'), field_dimen = (106.0,68.0), include_player_velocities=True, PlayerMarkerSize=10, PlayerAlpha=0.7)

Generating movie...done


## PPCF fig2movies

In [ ]:
# Creating png figures per slice
for i in range(start_frame-1,end_frame):
  # evaluated pitch control per frame
  print(i+1)
  PPCF,xgrid,ygrid = generate_pitch_control_for_track(i+1, tracking_home, tracking_away, params, GK_numbers, field_dimen = (106.,68.,), n_grid_cells_x = 50)
  plot_pitchcontrol_for_track( i+1, tracking_home, tracking_away, PPCF, annotate=False )

In [ ]:
# Merging all png figures to a movie
import cv2
import numpy as np

width=864
height=576
fps=25

# choose codec according to format needed
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('/content/PPCF_v1.avi', fourcc, fps, (width, height))


for i in range(start_frame-1,end_frame):
    img = cv2.imread('/content/PPCF_'+str(i+1)+'.png')
    video.write(img)

cv2.destroyAllWindows()
video.release()